# 0. Install and Import Dependencies

In [ ]:
# !pip install tensorflow-hub opencv-python matplotlib pandas shapely

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# add a center poinit
# --------------------------------------------

import numpy as np
import cv2
import pandas as pd
from pathlib import Path
import math

def get_angle(line1, line2):
    d1 = (line1[1][0] - line1[0][0], line1[1][1] - line1[0][1])
    d2 = (line2[1][0] - line2[0][0], line2[1][1] - line2[0][1])
    p = d1[0] * d2[0] + d1[1] * d2[1]
    n1 = math.sqrt(d1[0] * d1[0] + d1[1] * d1[1])
    n2 = math.sqrt(d2[0] * d2[0] + d2[1] * d2[1])
    ang = math.acos(p / (n1 * n2))
    ang = math.degrees(ang)
    return ang
def get_intersections(line1, line2):
    A = np.array(line1)
    B = np.array(line2)
    t, s = np.linalg.solve(np.array([A[1]-A[0], B[0]-B[1]]).T, B[0]-A[0])    
    
    return (1-t)*A[0] + t*A[1]
def getPerspectiveTransformMatrix(vid_path , show_frame = False):

    cap = cv2.VideoCapture(vid_path)
    # width , height = 1080 , 720
    border = np.zeros((720,1280,3), np.uint8)
    # print("Start")
    ret, frame = cap.read()

    test_frame =np.copy(frame)

    h, w = frame.shape[:2]
    mask = np.zeros([h+2, w+2], np.uint8) 
    # bgr
    diff_value = (3,1,3)
    black = (0,0,0)
    frame = cv2.GaussianBlur(frame , (7,7) , 0)

    fillpoints = [(100,0) , (100,700) , (300,350) , (1000,350) , (1200,700)]
    
    for point in fillpoints:
        cv2.floodFill(frame, mask, point, black, diff_value, diff_value)
    for point in fillpoints:
        cv2.circle(frame , point , 5 ,(0,0,255) , -1)

    # 灰階
    imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # 二值化
    ret,thresh = cv2.threshold(imgray,100,255,0)
    # 找輪廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contours = list(contours)
    # 根據輪廓面積大小進行sort
    contours.sort(key = cv2.contourArea , reverse=True)
    # 畫出最大的輪廓
    cv2.drawContours(border, contours[0:1], -1, (0,0,255), 10)



    # 灰階
    imgray = cv2.cvtColor(border,cv2.COLOR_BGR2GRAY)

    canny = cv2.Canny(imgray, 30, 100)
    lines = cv2.HoughLinesP(canny , 1.0 , np.pi/180 , 100 , np.array([]) , 200 , 100)

    cv2.line(frame,(100,100),(100,500),(255,255,255),5)

    horizon_line = []
    left_vertical_line = None
    right_vertical_line = None

    try:
    # 畫球場的垂直線 和 找到水平線座標
        for line in lines:
            for x1,y1,x2,y2 in line:
                line_angle = get_angle([(x1,y1),(x2,y2)] , [(0,0),(0,100)])
                line_angle_90 = 180 - line_angle if line_angle > 90 else line_angle
                vectorx = x2 - x1
                vectory = y2 - y1
                
                if  (line_angle_90 < 40 and int(line_angle)): 

                    if left_vertical_line == None and line_angle > 90:
                        # left_vertical_line = [(x1 - 20 , y1) , (x2 -20, y2)]
                        left_vertical_line = [(x1 , y1) , (x2, y2)]
                        cv2.line(frame,(x1 - vectorx * 100, y1 - vectory * 100),(x1 + vectorx * 100,y1 + vectory * 100),(255,0,0),5)
                    elif right_vertical_line == None and line_angle < 90:
                        right_vertical_line = [(x1, y1) , (x2, y2)]
                        cv2.line(frame,(x1 - vectorx * 100, y1 - vectory * 100),(x1 + vectorx * 100,y1 + vectory * 100),(255,0,0),5)

                elif line_angle_90 > 85:

                    horizon_line.append([[x1 ,y1] , [x2,y2] ])

        # 畫上下兩條水平線
        top_line = min(horizon_line , key = lambda x : x[0][1] + x[1][1])
        # top_line[0][1] -= 20
        # top_line[1][1] -= 20
        x1 , y1 = top_line[0]
        x2 , y2 = top_line[1]
        cv2.line(frame,(x1 - (x2-x1) * 100, y1 - (y2-y1) * 100),(x1 + (x2-x1) * 100,y1 + (y2-y1) * 100),(255,0,0),5)    
        bottom_line = max(horizon_line , key = lambda x : x[0][1] + x[1][1])
        # print(bottom_line)
        x1 , y1 = bottom_line[0]
        x2 , y2 = bottom_line[1]
        cv2.line(frame,(x1 - (x2-x1) * 100, y1 - (y2-y1) * 100),(x1 + (x2-x1) * 100,y1 + (y2-y1) * 100),(255,0,0),5)    

        # print(get_intersections(top_line , vertical_line[0]).astype(int))
        corner = []
        corner.append(get_intersections(top_line , left_vertical_line).astype(int))
        corner.append(get_intersections(bottom_line , left_vertical_line).astype(int))
        corner.append(get_intersections(bottom_line , right_vertical_line).astype(int))
        corner.append(get_intersections(top_line , right_vertical_line).astype(int))
        cv2.circle(frame , get_intersections(top_line , left_vertical_line).astype(int) , 5 , (0,255,0),-1)
        cv2.circle(frame , get_intersections(top_line , right_vertical_line).astype(int) , 5 , (0,255,0),-1)
        cv2.circle(frame , get_intersections(bottom_line , left_vertical_line).astype(int) , 5 , (0,255,0),-1)
        cv2.circle(frame , get_intersections(bottom_line , right_vertical_line).astype(int) , 5 , (0,255,0),-1)
        
    except:
        return -1 , -1 , -1, -1
        

    # cv2.imshow('board' , border)          
            
    # 進行透視變換
    old = np.float32(corner)
    new = np.float32([[0,0], [0,h-1], [w-1,h-1] , [w-1,0] ])
    matrix = cv2.getPerspectiveTransform(old , new)
    imgOutput = cv2.warpPerspective(test_frame, matrix, (w , h), cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))

    
    if show_frame:
        cv2.imshow('thresh' , thresh)     
        cv2.imshow('board' , border)  
        cv2.imshow('frame' , frame)
        cv2.imshow('Perspective', imgOutput)

        while cv2.waitKey(1) == -1:
            pass
    # cv2.destroyAllWindows()
    
    
    # Center point
    corner = np.array(corner)
    centerX = int(corner[:,0].mean())
    centerY = int(corner[:,1].mean())
    ratio = ((corner[1][1]-corner[0][1]+corner[2][1]-corner[3][1])/2)/(corner[2][0]-corner[1][0])
    tranY = int(centerY - ratio*115)
    center  = (centerX, tranY)
    

    return matrix , corner, center , 1

# 1. Load Model

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 2. Draw People

In [ ]:
def draw_keypoints(frame, shaped, confidence_threshold, color):
    for kp in shaped:
        ky, kx = kp
        cv2.circle(frame, (int(kx), int(ky)), 4, color, -1)

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_connections(frame, shaped, edges, confidence_threshold):
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1 = shaped[p1]
        y2, x2 = shaped[p2]
        cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,int(y2)), 4)

In [ ]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    i=126
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold, (0,i,0))
        i=255
    

# 3. Make Detections

In [ ]:

FILE_RES="/Users/sunny/Desktop/ai_badminton/part1/train/"  # 原影片的資料夾
FILE_DST="/Users/sunny/move_dataset/" # 要存 move_data 資料夾
CONFIDENCE_THRESHOLD = 0.1 # 判斷點的信心分數大於 CONFIDENCE_THRESHOLD
MIN_CONFIDENCE_NUM = 14 # 17個點至少要有 MIN_CONFIDENCE_NUM 的點大於 CONFIDENCE_THRESHOLD 才會拿 movnet預測的資料，否則拿前一幀的資料
SAVE_DATA = False # 要不要存檔案


In [ ]:
# detection and save A B all move data

for num in tqdm(range(472,801)):
    
    file_name = '%05d' % num
    df = pd.read_csv(f"{FILE_RES}{file_name}/{file_name}_S2.csv")
    cap = cv2.VideoCapture(f'{FILE_RES}{file_name}/{file_name}.mp4')
    matrix, corner, center, _ = getPerspectiveTransformMatrix(f'{FILE_RES}{file_name}/{file_name}.mp4')
    
    if center == -1:
        continue
        
    hitframe = 1
    index = 0

    y, x = 720, 1280

    A_player = []
    B_player = []
    A_label = []
    B_label = []

    while True:
        ret, frame = cap.read()


        if frame is None:
            break

        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640)
        input_img = tf.cast(img, dtype=tf.int32)


        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

        
        # Find the players
        for person in keypoints_with_scores:
            # y x
            shaped = np.squeeze(np.multiply(person, [y,x,1]))
            point = Point(shaped[16][1], shaped[16][0])
            polygon = Polygon(corner)

            if polygon.contains(point):

                if point.y > center[1] and (np.count_nonzero(shaped[:,2]>= CONFIDENCE_THRESHOLD)) > MIN_CONFIDENCE_NUM:
                    B_prev = shaped

                elif point.y <= center[1] and (np.count_nonzero(shaped[:,2] >= CONFIDENCE_THRESHOLD)) > MIN_CONFIDENCE_NUM:
                    A_prev = shaped

        A_player.append(A_prev)
        B_player.append(B_prev)
        
        
        if index < len(df):
            if hitframe == df.iloc[index]['HitFrame']-2 or hitframe == df.iloc[index]['HitFrame']-1 or hitframe == df.iloc[index]['HitFrame'] or hitframe == df.iloc[index]['HitFrame']+1 or hitframe == df.iloc[index]['HitFrame']+2:
                if df.iloc[index]['Hitter'] == 'A':
                    A_label.append(1)
                    B_label.append(0)
                else: 
                    A_label.append(0)
                    B_label.append(1)
                    
                if hitframe == df.iloc[index]['HitFrame']+2:
                    index+=1
            else:
                A_label.append(0)
                B_label.append(0)
        else:
            A_label.append(0)
            B_label.append(0)
            
    
        # Show the image
        
        # loop_through_people(frame, target, EDGES, 0.1)
        # print("draw")
        
        #A
        draw_connections(frame, A_prev[:,:2], EDGES, 0.1)
        draw_keypoints(frame, A_prev[:,:2], 0.1, (0,128,0))
        #B
        draw_connections(frame, B_prev[:,:2], EDGES, 0.1)
        draw_keypoints(frame, B_prev[:,:2], 0.1, (0,255,0))

        cv2.circle(frame , center, 5 , (0,0,0),-1)
        cv2.circle(frame , corner[0], 5 , (0,0,0),-1)
        cv2.circle(frame , corner[1], 5 , (0,0,0),-1)
        cv2.circle(frame , corner[2], 5 , (0,0,0),-1)
        cv2.circle(frame , corner[3], 5 , (0,0,0),-1)
        cv2.imshow('Movenet Multipose', frame)        

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
        hitframe += 1
            
    
    # Save data
    if SAVE_DATA:
        np.save(f'{FILE_DST}{file_name}_A_move', A_player)
        np.save(f'{FILE_DST}{file_name}_B_move', B_player)
        np.save(f'{FILE_DST}{file_name}_A_hit_label', A_label)
        np.save(f'{FILE_DST}{file_name}_B_hit_label', B_label)
    
    # print(hitframe)
cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# show A B all move data

for num in tqdm(range(1,801)):
    
    file_name = '%05d' % num
    
    if not os.path.exists(f'{FILE_DST}{file_name}_A_move.npy'):
        continue
    
    cap = cv2.VideoCapture(f'{FILE_RES}{file_name}/{file_name}.mp4')
    A_player = np.load(f'{FILE_DST}{file_name}_A_move.npy')
    B_player = np.load(f'{FILE_DST}{file_name}_B_move.npy')
        
    hitframe = 0
    index = 0

    while True:
        ret, frame = cap.read()


        if frame is None:
            break

    
        # Show the image
        
        # A
        draw_connections(frame, A_player[hitframe][:,:2], EDGES, 0.1)
        draw_keypoints(frame, A_player[hitframe][:,:2], 0.1, (0,128,0))
        
        # B
        draw_connections(frame, B_player[hitframe][:,:2], EDGES, 0.1)
        draw_keypoints(frame, B_player[hitframe][:,:2], 0.1, (0,255,0))
       
        cv2.imshow('Movenet Multipose', frame)        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
        hitframe += 1
        
    # print(hitframe)
    
cap.release()    
cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
# test
# train_A = np.load(f'/Users/sunny/move_dataset/{file_name}_A_move.npy')
# train_B = np.load(f'/Users/sunny/move_dataset/{file_name}_B_move.npy')
# label_A = np.load(f'/Users/sunny/move_dataset/{file_name}_A_hit_label.npy')
# label_B = np.load(f'/Users/sunny/move_dataset/{file_name}_B_hit_label.npy')
# print(label_A[:40])
# print(label_B[:40])

# print((train_A == A_player).all())
# print((train_B == B_player).all())
# print((label_A == A_label).all())
# print((label_B == B_label).all())

# print(np.array(A_player).shape)
# print(np.array(B_player).shape)

# print(A_label[100:140])
# print(B_label[100:140])

# Homography
# pts_src = np.array([[141, 131], [480, 159], [493, 630],[64, 601]])
# pts_dst = np.array([[0, 0],[256, 0],[256, 256],[0, 256]])
# h, status = cv2.findHomography(pts_src, pts_dst)
# im_out = cv2.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0]))

# load data

# for num in tqdm(range(1,801)):
#     # print(num)
#     file_name = '%05d' % num
#     if not os.path.exists(f'{FILE_DST}{file_name}{file_name}_A_move.npy'):
#         continue
        
#     A_player = np.load(f'{FILE_DST}{file_name}_A_move.npy')
#     B_player = np.load(f'{FILE_DST}{file_name}_B_move.npy')
    
